CNN: 0.98
ABC:

In [1]:
import torch
import torchvision
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.tools import dotdict
from driver.driver import ABC_Driver
torch.cuda.is_available()
torch.cuda.set_device(0)

In [2]:
mnist_args = dotdict()
mnist_args.train = dotdict()
mnist_args.predict = dotdict()

mnist_args.name = 'mnist'
mnist_args.train.batch_size = 100
mnist_args.predict.batch_size = 1000

In [16]:
args = dotdict()
args.data = mnist_args

args.train_epochs= 3
args.lr = 0.000001
args.criterion = 'CE'
args.use_gpu = True

In [17]:
driver = ABC_Driver(args)

Use GPU: cuda:None


In [18]:
driver.train()

epoch: 0, train_loss: 2.3124673255284627
epoch: 1, train_loss: 2.310705534617106
epoch: 2, train_loss: 2.3129204134146373


In [19]:
driver.metric()

0.1055

In [22]:
train_data = torchvision.datasets.MNIST('../../data/ABC/mnist', train=False, download=True,
                                                  transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                            torchvision.transforms.Normalize((0.1307,), (0.3081,))]))

In [23]:
train_data.

Dataset MNIST
    Number of datapoints: 10000
    Root location: ../../data/ABC/mnist
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [20]:
a = torch.tensor([[[1,2],[3,4]],[[1,2],[3,4]],[[1,2],[3,4]]])

In [21]:
torch.matmul(a,a)

tensor([[[ 7, 10],
         [15, 22]],

        [[ 7, 10],
         [15, 22]],

        [[ 7, 10],
         [15, 22]]])